In [ ]:
from llm import call_ai


print(call_ai(
    "You are a helpful assistant. Please answer the question: What is the capital of France?",
    'google'))

print(call_ai(
    "You are a helpful assistant. Please answer the question: What is the capital of France?",
    'openai'))

print(call_ai(
    "You are a helpful assistant. Please answer the question: What is the capital of France?",
    'ollama'))
        

The capital of France is **Paris**.
The capital of France is Paris.


In [1]:
%load_ext autoreload
%autoreload 
import templates
import loader  
from llm import call_ai
from datetime import datetime
from saver import save_user_imitation, save_evaluation_results, save_reflection_results
from logging_config import logger
#eval 
from eval import evaluate, evaluate_with_individual_scores

c:\Users\Christoph.Hau\Experimente\ha\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:


logger.info("Starting imitation generation...")
n= 3  # Number of stimuli to process, set to 3 for testing
#test_file_path = r'C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\534023.0.jsonl'
test_file_path = r'C:\Users\Christoph.Hau\Experimente\ha\data\raw\users\4252893976.0.jsonl'
run_id =  str(datetime.now().strftime("%Y%m%d_%H%M%S"))
user_history = loader.get_formatted_user_historie(test_file_path)

formatted_User_history = templates.format_template('persona_template_simple', **{'historie':user_history})
persona=call_ai(formatted_User_history, 'google')
print(f"Persona for user 534023:\n{persona}")
#load stimulus

#stimulus , is_post, post_id=loader.load_stimulus(test_file_path)
all_stimuli=loader.load_stimulus(test_file_path)
for x in all_stimuli[:n]:  # Limiting to first 3 stimuli for testing
    stimulus, is_post, post_id = x
    print(f"Stimulus: {stimulus}, is_post: {is_post}, post_id: {post_id}")
    if is_post:
        print(f"Stimulus for user 534023 is post:\n{stimulus}")
        stimulus_post = templates.format_template('imitation_post_template_simple', **{'persona': persona,'tweet': stimulus})
    else:
        print(f"Stimulus for user 534023 is reply:\n{stimulus}")
        stimulus_reply = templates.format_template('imitation_replies_template_simple', **{'persona':persona,'tweet': stimulus})

    #call ai with the stimulus post
    if is_post:
        imitation = call_ai(stimulus_post, 'ollama')
        print(f"Imitation for user 534023 post:\n{imitation}")
    else:
        imitation = call_ai(stimulus_reply, 'ollama')
        print(f"Imitation for user 534023 reply:\n{imitation}")

    #save imitation
    save_user_imitation(file_path=test_file_path, stimulus= stimulus,persona=persona,imitation=imitation,run_id= run_id, tweet_id=post_id)



print("Starting evaluation...")
results=loader.load_predictions_orginales_formated(run_id=run_id, file_path=test_file_path)
print(f"Results for run_id {run_id}:")
result=evaluate_with_individual_scores(results) #average evaluation of all results
save_evaluation_results(file_path=test_file_path, evaluation_results=result, run_id=run_id)
print(f"Evaluation results saved for run_id {run_id}: {result}")

results_dict=loader.load_results_for_reflection(run_id=run_id, file_path=test_file_path)
#templates.format_template("evaluation_template", **{'results': results_dict, 'run_id': run_id})
#reflection
iteration = 2
data = loader.load_results_for_reflection(run_id, test_file_path)
template= templates.format_template("reflect_results_template", **data)
improved_persona=call_ai(template, 'google_json')
try:
    save_reflection_results(file_path=test_file_path, run_id=run_id, reflections=improved_persona,iteration=iteration)
except Exception as e:
    logger.error(f"Error saving reflection results: {e}")
persona=loader.load_latest_improved_persona(run_id=run_id,file_path=test_file_path)


NameError: name 'logger' is not defined